# 모든 feature들 취합

# 1. 데이터 로드, 전처리

# 패키지 로드

In [1]:
# 패키지
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  itertools import combinations
from sklearn import linear_model
sns.set(palette='pastel')
sns.set(font_scale=2)

# labels

In [285]:
labels=pd.read_csv('FE and error_set/error_set_0.741.csv')

# train features

In [2]:
#### interaction sum에서 쓸만한 변수만 뽑음.
train_interaction_selected=pd.read_csv('wook_workspace/X_train_combination.csv')
#### stat, time_series, party, trade(동욱),trade(영빈), guild 
train_base_features=pd.read_csv('wook_workspace/X_train_base.csv')
train_activity_coint=pd.read_csv('X_train_counts.csv')
tain_party_variables=pd.read_csv('jimmy_workspace/train_party_variable.csv')
train_first_week=pd.read_csv('Features/train_first_week.csv')
train_first_week_act=pd.read_csv('Features/train_first_week_activity.csv')
train_pattern_partial_sum=pd.read_csv('Features/train_play_pattern_by_partail_sum.csv')
train_pattern_partial_sum_prob=pd.read_csv('Features/train_play_pattern_by_partail_sum_prob.csv')
train_playpattern_mean_encoded=pd.read_csv('Features/train_playpattern_mean_encoding.csv')
train_trade=pd.read_csv('Features/train_trade.csv')

FileNotFoundError: File b'wook_workspace/X_train_base.csv' does not exist

In [860]:
train_first_week_act=pd.read_csv('Features/train_first_week_activity.csv')

In [711]:
train_interaction_sum_every_features=pd.read_csv('Interactions/train_interaction_sum.csv')

# test features

In [857]:
#### interaction sum에서 쓸만한 변수만 뽑음.
test_interaction_selected=pd.read_csv('wook_workspace/X_test_combination.csv')
#### stat, time_series, party, trade(동욱),trade(영빈), guild 
test_base_features=pd.read_csv('wook_workspace/X_test_base.csv')
test_activity_coint=pd.read_csv('X_test_counts.csv')
test_party_variables=pd.read_csv('jimmy_workspace/test_party_variable_agg.csv')
test_first_week=pd.read_csv('Features/test_first_week.csv')
test_first_week_act=pd.read_csv('Features/test_first_week_activity.csv')
test_pattern_partial_sum=pd.read_csv('Features/test_play_pattern_by_partail_sum.csv')
test_pattern_partial_sum_prob=pd.read_csv('Features/test_play_pattern_by_partail_sum_prob.csv')
test_playpattern_mean_encoded=pd.read_csv('Features/test_playpattern_mean_encoding.csv')
test_trade=pd.read_csv('Features/test_trade.csv')

# merging

In [861]:
train_variable_list=[train_interaction_selected,train_base_features,train_activity_coint,tain_party_variables,
                     train_first_week,train_first_week_act,train_pattern_partial_sum,train_pattern_partial_sum_prob,
                     train_playpattern_mean_encoded,train_trade]

In [862]:
test_variable_list=[test_interaction_selected,test_base_features,test_activity_coint,tain_party_variables,
                     test_first_week,test_first_week_act,test_pattern_partial_sum,test_pattern_partial_sum_prob,
                     test_playpattern_mean_encoded,test_trade]

In [863]:
# 묶을 기준인 new_id가 있는지 확인

In [864]:
#new_id_exist?
train_new_id_exist=[]
for i in train_variable_list:
    train_new_id_exist.append('new_id' in i.columns)
    if train_new_id_exist[-1]==True:
        print(i.index)


RangeIndex(start=0, stop=100000, step=1)
RangeIndex(start=0, stop=100000, step=1)
RangeIndex(start=0, stop=100000, step=1)
RangeIndex(start=0, stop=100000, step=1)
RangeIndex(start=0, stop=100000, step=1)
RangeIndex(start=0, stop=100000, step=1)
RangeIndex(start=0, stop=100000, step=1)
RangeIndex(start=0, stop=100000, step=1)
RangeIndex(start=0, stop=100000, step=1)


In [865]:
test_new_id_exist=[]
for i in test_variable_list:
    test_new_id_exist.append('new_id' in i.columns)
    if test_new_id_exist[-1]==True:
        print(i.index)


RangeIndex(start=0, stop=40000, step=1)
RangeIndex(start=0, stop=40000, step=1)
RangeIndex(start=0, stop=100000, step=1)
RangeIndex(start=0, stop=40000, step=1)
RangeIndex(start=0, stop=40000, step=1)
RangeIndex(start=0, stop=40000, step=1)
RangeIndex(start=0, stop=40000, step=1)
RangeIndex(start=0, stop=40000, step=1)
RangeIndex(start=0, stop=40000, step=1)


# 중복된 열 처리

In [866]:
for i in range(len(train_variable_list)):
    if i ==0:
        train_concated=train_variable_list[i]
    elif train_new_id_exist[i] == True:
        train_concated=pd.merge(train_concated,train_variable_list[i],on='new_id',how='left')
    else:
        train_concated=pd.concat([train_concated,train_variable_list[i]],1)

In [867]:
for i in range(len(test_variable_list)):
    if i ==0:
        test_concated=test_variable_list[i]
    elif test_new_id_exist[i] == True:
        test_concated=pd.merge(test_concated,test_variable_list[i],on='new_id',how='left')
    else:
        test_concated=pd.concat([test_concated,test_variable_list[i]],1)

In [868]:
train_concated.columns[train_concated.columns.duplicated()]

Index([], dtype='object')

In [869]:
#train_concated_cc=train_concated.copy()
#test_concated_cc=test_concated.copy()
#train_concated=train_concated_cc
#test_concated=test_concated_cc

In [870]:
new_colname=list(train_concated.columns)
new_col_index=list(range(len(train_concated.columns)))
for i in train_concated.columns:
    if '_x' in i:
        new_colname.remove(i)
    if i == train_concated.columns[-1]:
        new_colname.remove("('npc_hongmun', 'item_hongmun').1")
        new_colname=new_colname[:-6]
        train_concated=train_concated.loc[:,new_colname]
        train_concated.columns=train_concated.columns.str.replace('_y','').tolist()

In [871]:
new_colname=list(test_concated.columns)
new_col_index=list(range(len(test_concated.columns)))
for i in test_concated.columns:
    if '_x' in i:
        new_colname.remove(i)
    if i == test_concated.columns[-1]:
        new_colname.remove("('npc_hongmun', 'item_hongmun').1")
        new_colname=new_colname[:-6]
        test_concated=test_concated.loc[:,new_colname]
        test_concated.columns=test_concated.columns.str.replace('_y','').tolist()

# 9월 5일 까지 만든 모든 데이터 concat 

In [873]:
list(test_concated.columns)

['new_id',
 'payment_amount_by_play_time',
 'play_time_by_cnt_dt',
 'game_combat_time_by_play_time',
 'get_money_by_play_time',
 'cnt_use_buffitem_by_game_combat_time',
 'chats_by_play_time',
 "('npc_exp', 'game_combat_time')",
 "('quest_exp', 'game_combat_time')",
 "('get_money', 'cnt_use_buffitem')",
 "('npc_hongmun', 'item_hongmun')",
 "('get_money', 'whisper_chat')",
 "('npc_exp', 'quest_exp')",
 'game_combat_time_by_cnt_dt',
 'npc_exp_by_cnt_dt',
 'quest_exp_by_cnt_dt',
 'npc_hongmun_by_cnt_dt',
 'quest_hongmun_by_cnt_dt',
 'npc_exp_by_play_time',
 'quest_exp_by_play_time',
 'npc_hongmun_by_play_time',
 'quest_hongmun_by_play_time',
 'payment_amount_min_stat',
 'payment_amount_max_stat',
 'payment_amount_range_stat',
 'payment_amount_median_stat',
 'payment_amount_sum_stat',
 'cnt_dt_min_stat',
 'cnt_dt_max_stat',
 'cnt_dt_range_stat',
 'cnt_dt_median_stat',
 'cnt_dt_sum_stat',
 'play_time_min_stat',
 'play_time_max_stat',
 'play_time_range_stat',
 'play_time_median_stat',
 'play_

In [874]:
del train_concated['8']
del test_concated['8']

# 여기 까지 concat 완료, 저장

In [18]:
train_concated.to_csv('Features/train_every_features.csv',index=False)
test_concated.to_csv('Features/test_every_features.csv',index=False)

NameError: name 'test_concated' is not defined

# 2. Ver1 추가

In [8]:
train_act_week=pd.read_csv('transformed/train_real_values_fillna.csv')
train_act_week=pd.read_csv('transformed/train_real_values_fillna.csv')

# 추가변수
'guild_chat_max_last_diff',
 'grocery_portion_in_market',
 'money_portion_in_market',
 'other_portion_in_market',
 'first_week_money_by_cnt_dt',
 'money_by_dt_max_last',
 'money_by_dt_first_last',
 'money_by_dt_first_max',
 'money_by_dt_median_last',
 'get_money_cnt_dt_lr',
 'first_week_money_by_play_time',
 'money_by_time_max_last',
 'money_by_time_first_last',
 'money_by_time_first_max',
 'money_by_time_median_last',
 'get_money_time_lr'

In [877]:
train_act_week=pd.read_csv('transformed/train_reaal_values_int.csv')
test_act_week=pd.read_csv('transformed/test_real_datas_int.csv')

In [10]:
train_act_last_week=train_act_week[train_act_week.wk.astype(int)==8]
#test_act_last_week=test_act_week[test_act_week.wk.astype(int)==8]

#  추가 1.get_money_by_day_train

In [773]:
train_act_week['get_money_by_cnt_dt']=train_act_week.get_money/train_act_week.cnt_dt

In [774]:
test_act_week['get_money_by_cnt_dt']=test_act_week.get_money/test_act_week.cnt_dt

In [775]:
money_cnt_dt=train_act_week.pivot('new_id','wk','get_money_by_cnt_dt')
money_cnt_dt=money_cnt_dt.reindex(index = money_cnt_dt.index.to_series().str.replace('tr','').astype(int).sort_values().index)

In [776]:
money_exist=((money_cnt_dt!=0)&(money_cnt_dt.isna()==False)).astype(int)
money_exist.head()

wk,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
new_id,,,,,,,,
tr0,0,0,0,0,0,0,0,0
tr1,0,0,0,0,0,0,0,1
tr2,0,0,0,0,1,1,0,1
tr3,0,0,0,0,0,0,0,1
tr4,1,1,1,1,1,1,1,1


In [777]:
for i in money_exist.columns:
    money_exist[i]=money_exist[i]*i
money_exist.head()

wk,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
new_id,,,,,,,,
tr0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tr1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
tr2,0.0,0.0,0.0,0.0,5.0,6.0,0.0,8.0
tr3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
tr4,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0


In [778]:
first_weeks_money=money_exist[(money_cnt_dt!=0)&(money_cnt_dt.isna()==False)].min(1)

In [779]:
first_weeks_money=pd.DataFrame(first_weeks_money)
first_weeks_money['new_id']=money_cnt_dt.index

In [780]:
first_weeks_money.head()

,0,new_id
new_id,,
tr0,NaN,tr0
tr1,8.0,tr1
tr2,5.0,tr2
tr3,8.0,tr3
tr4,1.0,tr4


In [781]:
train_act_week=pd.merge(train_act_week,first_weeks_ind,on='new_id',how='left')

In [782]:
first_weeks_money_value=train_act_week.loc[train_act_week.wk.astype(int)==train_act_week.iloc[:,-1],['new_id','get_money_by_cnt_dt']]
temp=list(first_weeks_money_value.columns)
temp[1]='first_week_money_by_cnt_dt'
first_weeks_money_value.columns=temp

In [783]:
money_cnt_dt=pd.merge(money_cnt_dt,first_weeks_money_value,on='new_id',how='left')

In [784]:
money_cnt_dt['money_by_dt_max_last']=money_cnt_dt[8]-money_cnt_dt.max(1)
money_cnt_dt['money_by_dt_first_last']=money_cnt_dt[8]-money_cnt_dt.first_week_money_by_cnt_dt
money_cnt_dt['money_by_dt_first_max']=money_cnt_dt.max(1)-money_cnt_dt.first_week_money_by_cnt_dt
money_cnt_dt['money_by_dt_median_last']=money_cnt_dt[8]-money_cnt_dt.median(1)


In [785]:
only_week_money=money_cnt_dt.loc[:,[1.0,
                                   2.0,                             3.0,
                                   4.0,                             5.0,
                                   6.0,                             7.0,
                                   8.0]].fillna(0)

In [786]:
only_week_money.index=money_cnt_dt.new_id

In [787]:
only_week_money.head()

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
new_id,,,,,,,,
tr0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
tr1,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,5437.0
tr2,0.000000,0.000000,0.0,0.000000,9436.000000,14457.714286,0.000000,19878.5
tr3,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,4671.5
tr4,255757.142857,782616.142857,541179.0,669924.142857,297650.285714,87077.500000,616011.714286,133752.0


In [788]:
#### 각 기간 별 선형회귀 fit

train_fit = dict()
for acc_id in only_week_money.index:
    lr = linear_model.LinearRegression(n_jobs = -1)
    lr.fit(only_week_money.columns.values.reshape(8,1),only_week_money.loc[only_week_money.index == acc_id,:].values.reshape(8,1))
    train_fit.update({acc_id:[lr.coef_, lr.intercept_] })
    
# fe_5 : trend - 장기 변화율(8주 선형회귀의 계수)
money_cnt_dt['get_money_cnt_dt_lr'] = money_cnt_dt.new_id.apply(lambda x : train_fit[x][0][0][0])

In [789]:
train_money_by_dt=money_cnt_dt.loc[:,['new_id','first_week_money_by_cnt_dt',
             'money_by_dt_max_last',     'money_by_dt_first_last',
            'money_by_dt_first_max',    'money_by_dt_median_last','get_money_cnt_dt_lr']]
train_money_by_dt.to_csv('Features/train_money_by_dt.csv',index=False)

#  추가 2.get_money_by_day_test

In [791]:
money_cnt_dt=test_act_week.pivot('new_id','wk','get_money_by_cnt_dt')
money_cnt_dt=money_cnt_dt.reindex(index = money_cnt_dt.index.to_series().str.replace('te','').astype(int).sort_values().index)

In [792]:
money_exist=((money_cnt_dt!=0)&(money_cnt_dt.isna()==False)).astype(int)
money_exist.head()

wk,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
new_id,,,,,,,,
te0,1,1,1,1,1,1,1,1
te1,1,1,1,1,1,1,1,1
te2,0,0,0,0,0,0,0,0
te3,0,0,0,0,0,1,1,1
te4,0,0,0,0,0,0,0,0


In [793]:
for i in money_exist.columns:
    money_exist[i]=money_exist[i]*i
money_exist.head()

wk,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
new_id,,,,,,,,
te0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
te1,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
te2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
te3,0.0,0.0,0.0,0.0,0.0,6.0,7.0,8.0
te4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [794]:
first_weeks_money=money_exist[(money_cnt_dt!=0)&(money_cnt_dt.isna()==False)].min(1)

In [795]:
first_weeks_money['new_id']=money_cnt_dt.index

In [796]:
first_weeks_money=money_exist[(money_cnt_dt!=0)&(money_cnt_dt.isna()==False)].min(1)

In [797]:
first_weeks_money=pd.DataFrame(first_weeks_money)
first_weeks_money['new_id']=money_cnt_dt.index

In [798]:
first_weeks_money.head()

,0,new_id
new_id,,
te0,1.0,te0
te1,1.0,te1
te2,NaN,te2
te3,6.0,te3
te4,NaN,te4


In [799]:
test_act_week=pd.merge(test_act_week,first_weeks_money,on='new_id',how='left')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'new_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [800]:
first_weeks_money_value=test_act_week.loc[test_act_week.wk.astype(int)==test_act_week.iloc[:,-1],['new_id','get_money_by_cnt_dt']]
temp=list(first_weeks_money_value.columns)
temp[1]='first_week_money_by_cnt_dt'
first_weeks_money_value.columns=temp

In [801]:
money_cnt_dt=pd.merge(money_cnt_dt,first_weeks_money_value,on='new_id',how='left')

In [802]:
money_cnt_dt['money_by_dt_max_last']=money_cnt_dt[8]-money_cnt_dt.max(1)
money_cnt_dt['money_by_dt_first_last']=money_cnt_dt[8]-money_cnt_dt.first_week_money_by_cnt_dt
money_cnt_dt['money_by_dt_first_max']=money_cnt_dt.max(1)-money_cnt_dt.first_week_money_by_cnt_dt
money_cnt_dt['money_by_dt_median_last']=money_cnt_dt[8]-money_cnt_dt.median(1)


In [803]:
money_cnt_dt.head(10)

,new_id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,first_week_money_by_cnt_dt,money_by_dt_max_last,money_by_dt_first_last,money_by_dt_first_max,money_by_dt_median_last
0,te0,7.180517e+06,543276.600000,596199.500000,4.536650e+04,2.525191e+06,22594.500000,814342.000000,6.115946e+06,7.180517e+06,-1.064571e+06,-1.064571e+06,0.00,5.546208e+06
1,te1,1.354653e+06,292069.333333,726894.800000,1.185102e+05,1.248549e+06,715760.285714,275874.500000,7.541880e+05,1.354653e+06,-6.004646e+05,-6.004646e+05,0.00,2.502732e+05
2,te2,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000e+00,NaN,0.000000e+00,NaN,NaN,0.000000e+00
3,te3,NaN,NaN,NaN,NaN,NaN,3807.500000,3999.666667,1.049050e+04,3.807500e+03,0.000000e+00,6.683000e+03,6683.00,6.490833e+03
4,te4,NaN,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,NaN,0.000000e+00,NaN,NaN,0.000000e+00
5,te5,1.277000e+03,2015.166667,1820.000000,1.883500e+03,1.811600e+03,1626.833333,1510.666667,2.554750e+03,1.277000e+03,0.000000e+00,1.277750e+03,1277.75,9.860000e+02
6,te6,NaN,NaN,13572.000000,1.423500e+03,NaN,NaN,0.000000,1.166667e+02,1.357200e+04,-1.345533e+04,-1.345533e+04,0.00,5.833333e+01
7,te7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,0.000000e+00,NaN,NaN,0.000000e+00
8,te8,NaN,3311.500000,866.000000,1.785000e+03,9.850000e+02,NaN,680835.400000,3.221000e+03,3.311500e+03,-6.776144e+05,-9.050000e+01,677523.90,7.180000e+02
9,te9,3.768743e+05,356377.714286,316809.571429,3.306552e+06,1.443886e+06,109640.714286,380648.000000,3.499657e+05,3.768743e+05,-2.956587e+06,-2.690857e+04,2929678.00,-1.666029e+04


In [804]:
only_week_money=money_cnt_dt.loc[:,[1.0,
                                   2.0,                             3.0,
                                   4.0,                             5.0,
                                   6.0,                             7.0,
                                   8.0]].fillna(0)

In [805]:
only_week_money.index=money_cnt_dt.new_id

In [806]:
only_week_money.head()

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
new_id,,,,,,,,
te0,7180517.0,543276.600000,596199.5,45366.5,2.525191e+06,22594.500000,814342.000000,6115946.0
te1,1354652.6,292069.333333,726894.8,118510.2,1.248549e+06,715760.285714,275874.500000,754188.0
te2,0.0,0.000000,0.0,0.0,0.000000e+00,0.000000,0.000000,0.0
te3,0.0,0.000000,0.0,0.0,0.000000e+00,3807.500000,3999.666667,10490.5
te4,0.0,0.000000,0.0,0.0,0.000000e+00,0.000000,0.000000,0.0


In [807]:
#### 각 기간 별 선형회귀 fit

train_fit = dict()
for acc_id in only_week_money.index:
    lr = linear_model.LinearRegression(n_jobs = -1)
    lr.fit(only_week_money.columns.values.reshape(8,1),only_week_money.loc[only_week_money.index == acc_id,:].values.reshape(8,1))
    train_fit.update({acc_id:[lr.coef_, lr.intercept_] })
    
# fe_5 : trend - 장기 변화율(8주 선형회귀의 계수)
money_cnt_dt['get_money_cnt_dt_lr'] = money_cnt_dt.new_id.apply(lambda x : train_fit[x][0][0][0])

In [808]:
test_money_by_dt=money_cnt_dt.loc[:,['new_id','first_week_money_by_cnt_dt',
             'money_by_dt_max_last',     'money_by_dt_first_last',
            'money_by_dt_first_max',    'money_by_dt_median_last','get_money_cnt_dt_lr']]
test_money_by_dt.to_csv('Features/test_money_by_dt.csv',index=False)

#  추가 3.get_money_by_day_test

In [810]:
train_act_week['get_money_by_play_time']=train_act_week.get_money/train_act_week.play_time

In [811]:
test_act_week['get_money_by_play_time']=test_act_week.get_money/test_act_week.play_time

In [812]:
money_play_time=train_act_week.pivot('new_id','wk','get_money_by_play_time')
money_play_time=money_play_time.reindex(index = money_play_time.index.to_series().str.replace('tr','').astype(int).sort_values().index)

In [813]:
money_exist=((money_play_time!=0)&(money_play_time.isna()==False)).astype(int)
money_exist.head()

wk,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
new_id,,,,,,,,
tr0,0,0,0,0,0,0,0,0
tr1,0,0,0,0,0,0,0,1
tr2,0,0,0,0,1,1,0,1
tr3,0,0,0,0,0,0,0,1
tr4,1,1,1,1,1,1,1,1


In [814]:
for i in money_exist.columns:
    money_exist[i]=money_exist[i]*i
money_exist.head()

wk,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
new_id,,,,,,,,
tr0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
tr1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
tr2,0.0,0.0,0.0,0.0,5.0,6.0,0.0,8.0
tr3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
tr4,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0


In [815]:
first_weeks_money=money_exist[(money_play_time!=0)&(money_play_time.isna()==False)].min(1)

In [816]:
first_weeks_money=pd.DataFrame(first_weeks_money)
first_weeks_money['new_id']=money_play_time.index

In [817]:
first_weeks_money.head()

,0,new_id
new_id,,
tr0,NaN,tr0
tr1,8.0,tr1
tr2,5.0,tr2
tr3,8.0,tr3
tr4,1.0,tr4


In [818]:
train_act_week=pd.merge(train_act_week,first_weeks_ind,on='new_id',how='left')

In [819]:
first_weeks_money_value=train_act_week.loc[train_act_week.wk.astype(int)==train_act_week.iloc[:,-1],['new_id','get_money_by_play_time']]
temp=list(first_weeks_money_value.columns)
temp[1]='first_week_money_by_play_time'
first_weeks_money_value.columns=temp

In [820]:
money_play_time=pd.merge(money_play_time,first_weeks_money_value,on='new_id',how='left')

In [821]:
money_play_time['money_by_time_max_last']=money_play_time[8]-money_play_time.max(1)
money_play_time['money_by_time_first_last']=money_play_time[8]-money_play_time.first_week_money_by_play_time
money_play_time['money_by_time_first_max']=money_play_time.max(1)-money_play_time.first_week_money_by_play_time
money_play_time['money_by_time_median_last']=money_play_time[8]-money_play_time.median(1)


In [822]:
money_play_time.head()

,new_id,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,first_week_money_by_play_time,money_by_time_max_last,money_by_time_first_last,money_by_time_first_max,money_by_time_median_last
0,tr0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,NaN,NaN,0.000000
1,tr1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.136242,0.136242,0.000000,0.000000,0.000000,0.136242
2,tr2,NaN,NaN,NaN,NaN,0.165960,0.216677,NaN,0.566396,0.165960,0.000000,0.400435,0.400435,0.349718
3,tr3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.130834,0.130834,0.000000,0.000000,0.000000,0.130834
4,tr4,12.584969,23.778948,18.363103,18.476588,14.037932,14.953205,34.205499,9.391566,12.584969,-24.813933,-3.193403,21.620529,-5.104003


In [823]:
only_week_money=money_play_time.loc[:,[1.0,
                                   2.0,                             3.0,
                                   4.0,                             5.0,
                                   6.0,                             7.0,
                                   8.0]].fillna(0)

In [824]:
only_week_money.index=money_play_time.new_id

In [825]:
only_week_money.head()

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
new_id,,,,,,,,
tr0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
tr1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.136242
tr2,0.000000,0.000000,0.000000,0.000000,0.165960,0.216677,0.000000,0.566396
tr3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.130834
tr4,12.584969,23.778948,18.363103,18.476588,14.037932,14.953205,34.205499,9.391566


In [826]:
#### 각 기간 별 선형회귀 fit

train_fit = dict()
for acc_id in only_week_money.index:
    lr = linear_model.LinearRegression(n_jobs = -1)
    lr.fit(only_week_money.columns.values.reshape(8,1),only_week_money.loc[only_week_money.index == acc_id,:].values.reshape(8,1))
    train_fit.update({acc_id:[lr.coef_, lr.intercept_] })
    
# fe_5 : trend - 장기 변화율(8주 선형회귀의 계수)
money_play_time['get_money_time_lr'] = money_play_time.new_id.apply(lambda x : train_fit[x][0][0][0])

In [827]:
train_money_by_time=money_play_time.loc[:,['new_id','first_week_money_by_play_time',
             'money_by_time_max_last',     'money_by_time_first_last',
            'money_by_time_first_max',    'money_by_time_median_last','get_money_time_lr']]
train_money_by_time.to_csv('Features/train_money_by_time.csv',index=False)

# 추가3.get_money_by_day_test

In [829]:
money_play_time=test_act_week.pivot('new_id','wk','get_money_by_play_time')
money_play_time=money_play_time.reindex(index = money_play_time.index.to_series().str.replace('te','').astype(int).sort_values().index)

In [830]:
money_exist=((money_play_time!=0)&(money_play_time.isna()==False)).astype(int)
money_exist.head()

wk,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
new_id,,,,,,,,
te0,1,1,1,1,1,1,1,1
te1,1,1,1,1,1,1,1,1
te2,0,0,0,0,0,0,0,0
te3,0,0,0,0,0,1,1,1
te4,0,0,0,0,0,0,0,0


In [831]:
for i in money_exist.columns:
    money_exist[i]=money_exist[i]*i
money_exist.head()

wk,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
new_id,,,,,,,,
te0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
te1,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
te2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
te3,0.0,0.0,0.0,0.0,0.0,6.0,7.0,8.0
te4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [832]:
first_weeks_money=money_exist[(money_play_time!=0)&(money_play_time.isna()==False)].min(1)

In [833]:
first_weeks_money['new_id']=money_play_time.index

In [834]:
first_weeks_money=money_exist[(money_play_time!=0)&(money_play_time.isna()==False)].min(1)

In [835]:
first_weeks_money=pd.DataFrame(first_weeks_money)
first_weeks_money['new_id']=money_play_time.index

In [836]:
first_weeks_money.head()

,0,new_id
new_id,,
te0,1.0,te0
te1,1.0,te1
te2,NaN,te2
te3,6.0,te3
te4,NaN,te4


In [837]:
test_act_week=pd.merge(test_act_week,first_weeks_money,on='new_id',how='left')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 'new_id' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


In [838]:
first_weeks_money_value=test_act_week.loc[test_act_week.wk.astype(int)==test_act_week.iloc[:,-1],['new_id','get_money_by_play_time']]
temp=list(first_weeks_money_value.columns)
temp[1]='first_week_money_by_play_time'
first_weeks_money_value.columns=temp

In [839]:
money_play_time=pd.merge(money_play_time,first_weeks_money_value,on='new_id',how='left')

In [840]:
money_play_time['money_by_time_max_last']=money_play_time[8]-money_play_time.max(1)
money_play_time['money_by_time_first_last']=money_play_time[8]-money_play_time.first_week_money_by_play_time
money_play_time['money_by_time_first_max']=money_play_time.max(1)-money_play_time.first_week_money_by_play_time
money_play_time['money_by_time_median_last']=money_play_time[8]-money_play_time.median(1)


In [841]:
only_week_money=money_play_time.loc[:,[1.0,
                                   2.0,                             3.0,
                                   4.0,                             5.0,
                                   6.0,                             7.0,
                                   8.0]].fillna(0)

In [842]:
only_week_money.index=money_play_time.new_id

In [843]:
#### 각 기간 별 선형회귀 fit

train_fit = dict()
for acc_id in only_week_money.index:
    lr = linear_model.LinearRegression(n_jobs = -1)
    lr.fit(only_week_money.columns.values.reshape(8,1),only_week_money.loc[only_week_money.index == acc_id,:].values.reshape(8,1))
    train_fit.update({acc_id:[lr.coef_, lr.intercept_] })
    
# fe_5 : trend - 장기 변화율(8주 선형회귀의 계수)
money_play_time['get_money_time_lr'] = money_play_time.new_id.apply(lambda x : train_fit[x][0][0][0])

In [844]:
test_money_by_time=money_play_time.loc[:,['new_id','first_week_money_by_play_time',
             'money_by_time_max_last',     'money_by_time_first_last',
            'money_by_time_first_max',    'money_by_time_median_last','get_money_time_lr']]
test_money_by_time.to_csv('Features/test_money_by_time.csv',index=False)

,cnt_dt,play_time,npc_exp,npc_hongmun,quest_exp,quest_hongmun,item_hongmun,game_combat_time,get_money,duel_cnt,...,district_chat,party_chat,guild_chat,faction_chat,cnt_use_buffitem,gathering_cnt,making_cnt,payment_amount,wk,new_id
0,1.0,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8.0,tr0
1,1.0,39907,46489,0,971470,0,0,9221,5437,0,...,0,0,0,0,105,0,0,0,8.0,tr1
2,3.0,170571,124717,39093,5004230,8456830,0,49160,28308,0,...,0,25,0,0,299,0,0,0,5.0,tr2
3,7.0,467073,313170,194542,16352370,33827320,0,148303,101204,0,...,0,60,0,0,916,0,0,0,6.0,tr2
4,2.0,70193,0,226222,0,715840,114,34395,39757,0,...,0,45,0,0,160,0,0,0,8.0,tr2
5,2.0,71411,45205,7532,1709340,2190590,0,16680,9343,0,...,0,27,0,0,167,0,0,0,8.0,tr3
6,7.0,142257,312,1144989,427530,3833812,1060,30658,1790300,0,...,19,481,2288,0,166,0,0,178200,1.0,tr4
7,7.0,230385,3432,1804710,215050,8913079,764,49847,5478313,0,...,150,1039,1962,0,247,0,7,3960,2.0,tr4
8,7.0,206297,0,835441,0,3893668,450,40185,3788253,0,...,46,1152,1573,0,200,0,14,69300,3.0,tr4
9,7.0,253806,7420,647390,1946200,10031148,357,45810,4689469,0,...,21,1148,1514,0,314,0,35,0,4.0,tr4


#  추가 4. guild chat max - guildchat_last 

In [997]:
train_concated_0906['guild_chat_max_last_diff']=train_concated_0906['guild_chat_max_stat'].fillna(0)-train_act_last_week['guild_chat'].fillna(0)

In [996]:
test_concated_0906['guild_chat_max_last_diff']=test_concated_0906['guild_chat_max_stat'].fillna(0)-test_act_last_week['guild_chat'].fillna(0)

In [1001]:
train_act_last_week.guild_chat

0           0
1           0
4           0
5           0
13        585
19          0
23          0
29          0
32          0
36          0
38          0
44          0
47          0
57          0
60          0
63          0
64          0
68          0
69          0
70        616
78          0
79          0
87          0
92          0
96          0
98          0
99          0
100         0
101         0
102         0
         ... 
440444      0
440452    572
440453      0
440454      0
440456      0
440462      0
440464      0
440469      0
440480      0
440483      0
440491      0
440494      0
440495      0
440496      0
440498      0
440501      0
440511      0
440517      0
440531      0
440532      0
440533      0
440534      0
440540      0
440544    348
440556    440
440560      0
440563      0
440568      0
440574    412
440577      0
Name: guild_chat, Length: 100000, dtype: int64

#  추가 5.품목별 전체 거래량 대비 개인 거래량

In [409]:
#money
train_concated_0906['money_portion_in_market']=(train_concated_0906.loc[:,['give_money_sum','rec_money_sum','give_exc_money_sum']]/(train_concated_0906.loc[:,['give_money_sum','rec_money_sum','give_exc_money_sum']].sum()+test_concated_0906.loc[:,['give_money_sum','rec_money_sum','give_exc_money_sum']].sum())).sum(1)
test_concated_0906['money_portion_in_market']=(test_concated_0906.loc[:,['give_money_sum','rec_money_sum','give_exc_money_sum']]/(train_concated_0906.loc[:,['give_money_sum','rec_money_sum','give_exc_money_sum']].sum()+test_concated_0906.loc[:,['give_money_sum','rec_money_sum','give_exc_money_sum']].sum())).sum(1)

In [410]:
#grocery
train_concated_0906['grocery_portion_in_market']=(train_concated_0906.loc[:,['give_grocery_sum','rec_grocery_sum','give_exc_grocery_sum']]/(train_concated_0906.loc[:,['give_grocery_sum','rec_grocery_sum','give_exc_grocery_sum']].sum()+test_concated_0906.loc[:,['give_grocery_sum','rec_grocery_sum','give_exc_grocery_sum']].sum())).sum(1)
test_concated_0906['grocery_portion_in_market']=(test_concated_0906.loc[:,['give_grocery_sum','rec_grocery_sum','give_exc_grocery_sum']]/(train_concated_0906.loc[:,['give_grocery_sum','rec_grocery_sum','give_exc_grocery_sum']].sum()+test_concated_0906.loc[:,['give_grocery_sum','rec_grocery_sum','give_exc_grocery_sum']].sum())).sum(1)

In [412]:
#money grocery를 제외한 나머지 재화들에 대한 거래량을 기록한 colname들
others_colname=[x for x in train_concated_0906.columns if ('sum' in x)&((('weapon') in x)|(('acc') in x)|(('gem') in x)|(('costume') in x))]


In [413]:
#others
train_concated_0906['other_portion_in_market']=(train_concated_0906.loc[:,others_colname]/(train_concated_0906.loc[:,others_colname].sum()+test_concated_0906.loc[:,others_colname].sum())).sum(1)
test_concated_0906['other_portion_in_market']=(test_concated_0906.loc[:,others_colname]/(train_concated_0906.loc[:,others_colname].sum()+test_concated_0906.loc[:,others_colname].sum())).sum(1)

In [846]:
pd.read_csv('Features/train_trade.csv')

,new_id,source_cnt,target_cnt,exchange_cnt,from_how_many_cnt,to_how_many_cnt,with_how_many_cnt,give_gini,rec_gini,exc_gini,...,exc_last,give_first,give_second,give_third,rec_first,rec_second,rec_third,exc_first,exc_second,exc_third
0,tr0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tr1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tr2,8.0,NaN,NaN,NaN,1.0,NaN,0.000000,NaN,NaN,...,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tr3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,tr4,18.0,14.0,14.0,11.0,8.0,14.0,0.808642,0.890625,0.928571,...,7.0,1.0,2.0,5.0,5.0,3.0,1.0,2.0,3.0,1.0
5,tr5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,tr6,NaN,5.0,5.0,NaN,NaN,3.0,NaN,NaN,0.640000,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,7.0,NaN
7,tr7,1.0,NaN,NaN,NaN,1.0,NaN,0.000000,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,tr8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,tr9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#  추가 6.일별, 플레이타임별, 전투시간별 거래건수

In [903]:
train_concated_0906['trade_cnt_by_cnt_dt']=(train_concated_0906.source_cnt.fillna(0)+train_concated_0906.target_cnt.fillna(0)+train_concated_0906.exc_cnt.fillna(0))/train_concated_0906.cnt_dt_sum_stat
train_concated_0906['trade_cnt_by_play_time']=(train_concated_0906.source_cnt.fillna(0)+train_concated_0906.target_cnt.fillna(0)+train_concated_0906.exc_cnt.fillna(0))/train_concated_0906.play_time_sum_stat
train_concated_0906['trade_cnt_by_combat_time']=(train_concated_0906.source_cnt.fillna(0)+train_concated_0906.target_cnt.fillna(0)+train_concated_0906.exc_cnt.fillna(0))/train_concated_0906.game_combat_time_sum_stat

In [905]:
test_concated_0906['trade_cnt_by_cnt_dt']=(test_concated_0906.source_cnt.fillna(0)+test_concated_0906.target_cnt.fillna(0)+test_concated_0906.exc_cnt.fillna(0))/test_concated_0906.cnt_dt_sum_stat
test_concated_0906['trade_cnt_by_play_time']=(test_concated_0906.source_cnt.fillna(0)+test_concated_0906.target_cnt.fillna(0)+test_concated_0906.exc_cnt.fillna(0))/test_concated_0906.play_time_sum_stat
test_concated_0906['trade_cnt_by_combat_time']=(test_concated_0906.source_cnt.fillna(0)+test_concated_0906.target_cnt.fillna(0)+test_concated_0906.exc_cnt.fillna(0))/test_concated_0906.game_combat_time_sum_stat

#  추가 7. 추가사항 취합

In [851]:
train_concated_0906=pd.merge(train_concated_0906,train_money_by_dt,on='new_id',how='left')
train_concated_0906=pd.merge(train_concated_0906,train_money_by_time,on='new_id',how='left')

In [852]:
test_concated_0906=pd.merge(test_concated_0906,test_money_by_dt,on='new_id',how='left')
test_concated_0906=pd.merge(test_concated_0906,test_money_by_time,on='new_id',how='left')

# 추가 8. 제거할 변수 제거 후 저장

In [934]:
train_concated_0906=train_concated_0906.loc[:,train_concated_0906.columns.duplicated()==False]
test_concated_0906=test_concated_0906.loc[:,test_concated_0906.columns.duplicated()==False]

In [976]:
train_concated_0906.columns=train_concated_0906.columns.str.replace(',','_product_')
train_concated_0906.columns=train_concated_0906.columns.str.replace("'",'')

In [977]:
test_concated_0906.columns=test_concated_0906.columns.str.replace(',','_product_')
test_concated_0906.columns=test_concated_0906.columns.str.replace("'",'')

In [978]:
list(train_concated_0906.columns)

['new_id',
 'payment_amount_by_play_time',
 'play_time_by_cnt_dt',
 'game_combat_time_by_play_time',
 'get_money_by_play_time',
 'cnt_use_buffitem_by_game_combat_time',
 'chats_by_play_time',
 '(npc_exp_product_ game_combat_time)',
 '(quest_exp_product_ game_combat_time)',
 '(get_money_product_ cnt_use_buffitem)',
 '(npc_hongmun_product_ item_hongmun)',
 '(get_money_product_ whisper_chat)',
 '(npc_exp_product_ quest_exp)',
 'game_combat_time_by_cnt_dt',
 'npc_exp_by_cnt_dt',
 'quest_exp_by_cnt_dt',
 'npc_hongmun_by_cnt_dt',
 'quest_hongmun_by_cnt_dt',
 'npc_exp_by_play_time',
 'quest_exp_by_play_time',
 'npc_hongmun_by_play_time',
 'quest_hongmun_by_play_time',
 'payment_amount_min_stat',
 'payment_amount_max_stat',
 'payment_amount_range_stat',
 'payment_amount_median_stat',
 'payment_amount_sum_stat',
 'cnt_dt_min_stat',
 'cnt_dt_max_stat',
 'cnt_dt_range_stat',
 'cnt_dt_median_stat',
 'cnt_dt_sum_stat',
 'play_time_min_stat',
 'play_time_max_stat',
 'play_time_range_stat',
 'play_ti

In [941]:
train_trade=pd.read_csv('Features/train_trade.csv')
test_trade=pd.read_csv('Features/test_trade.csv')

In [982]:
train_concated_0906=pd.concat([train_concated_0906,train_trade.loc[:,[x for x in train_trade.columns if x not in train_concated_0906.columns]]],1)
train_concated_0906=train_concated_0906.loc[:,[x for x in train_concated_0906.columns if '_x' not in x]]
train_concated_0906.columns=train_concated_0906.columns.str.replace('_y','')

In [984]:
test_concated_0906=pd.concat([test_concated_0906,test_trade.loc[:,[x for x in test_trade.columns if x not in test_concated_0906.columns]]],1)
test_concated_0906=test_concated_0906.loc[:,[x for x in test_concated_0906.columns if '_x' not in x]]
test_concated_0906.columns=test_concated_0906.columns.str.replace('_y','')

In [985]:
list(train_concated_0906.columns)

['new_id',
 'payment_amount_by_play_time',
 'play_time_by_cnt_dt',
 'game_combat_time_by_play_time',
 'get_money_by_play_time',
 'cnt_use_buffitem_by_game_combat_time',
 'chats_by_play_time',
 '(npc_exp_product_ game_combat_time)',
 '(quest_exp_product_ game_combat_time)',
 '(get_money_product_ cnt_use_buffitem)',
 '(npc_hongmun_product_ item_hongmun)',
 '(get_money_product_ whisper_chat)',
 '(npc_exp_product_ quest_exp)',
 'game_combat_time_by_cnt_dt',
 'npc_exp_by_cnt_dt',
 'quest_exp_by_cnt_dt',
 'npc_hongmun_by_cnt_dt',
 'quest_hongmun_by_cnt_dt',
 'npc_exp_by_play_time',
 'quest_exp_by_play_time',
 'npc_hongmun_by_play_time',
 'quest_hongmun_by_play_time',
 'payment_amount_min_stat',
 'payment_amount_max_stat',
 'payment_amount_range_stat',
 'payment_amount_median_stat',
 'payment_amount_sum_stat',
 'cnt_dt_min_stat',
 'cnt_dt_max_stat',
 'cnt_dt_range_stat',
 'cnt_dt_median_stat',
 'cnt_dt_sum_stat',
 'play_time_min_stat',
 'play_time_max_stat',
 'play_time_range_stat',
 'play_ti

In [986]:
del train_concated_0906['8']
del test_concated_0906['8']

In [989]:
list(train_concated_0906.columns)

['new_id',
 'payment_amount_by_play_time',
 'play_time_by_cnt_dt',
 'game_combat_time_by_play_time',
 'get_money_by_play_time',
 'cnt_use_buffitem_by_game_combat_time',
 'chats_by_play_time',
 '(npc_exp_product_ game_combat_time)',
 '(quest_exp_product_ game_combat_time)',
 '(get_money_product_ cnt_use_buffitem)',
 '(npc_hongmun_product_ item_hongmun)',
 '(get_money_product_ whisper_chat)',
 '(npc_exp_product_ quest_exp)',
 'game_combat_time_by_cnt_dt',
 'npc_exp_by_cnt_dt',
 'quest_exp_by_cnt_dt',
 'npc_hongmun_by_cnt_dt',
 'quest_hongmun_by_cnt_dt',
 'npc_exp_by_play_time',
 'quest_exp_by_play_time',
 'npc_hongmun_by_play_time',
 'quest_hongmun_by_play_time',
 'payment_amount_min_stat',
 'payment_amount_max_stat',
 'payment_amount_range_stat',
 'payment_amount_median_stat',
 'payment_amount_sum_stat',
 'cnt_dt_min_stat',
 'cnt_dt_max_stat',
 'cnt_dt_range_stat',
 'cnt_dt_median_stat',
 'cnt_dt_sum_stat',
 'play_time_min_stat',
 'play_time_max_stat',
 'play_time_range_stat',
 'play_ti

In [1004]:
train_concated_0906.loc[:,train_concated_0906.columns.duplicated()==False].to_csv('Features/train_every_features_0906.csv',index=False)
test_concated_0906.loc[:,test_concated_0906.columns.duplicated()==False].to_csv('Features/test_every_features_0906.csv',index=False)

In [1002]:
train_concated_0906.loc[:,['new_id','guild_chat_max_last_diff','give_gini',
 'rec_gini',
 'exc_gini','source+exc',
 'target+exc',
 'exc_ratio',
 'one_way_ratio',
 'first_trade_all',
 'grocery_portion_in_market',
 'money_portion_in_market',
 'other_portion_in_market',
 'first_week_money_by_cnt_dt',
 'money_by_dt_first_max',
 'get_money_cnt_dt_lr',
 'trade_cnt_by_cnt_dt',
 'trade_cnt_by_play_time',
 'trade_cnt_by_combat_time',
 'first_week_money_by_play_time',
 'money_by_time_max_last',
 'money_by_time_first_last',
 'money_by_time_first_max',
 'money_by_time_median_last',
 'get_money_time_lr',
 'last_trade_all',
 'last_week_trade_cnt',
 'first_week_trade_cnt',
 'weekly_max_trade_cnt',
 'weekly_trade_cnt_std',
 'weekly_trade_cycle',
 'trade_first-last',
 'trade_last-max']].to_csv('Features/train_0906_added.csv',index=False)

In [1007]:
test_concated_0906.loc[:,['new_id','guild_chat_max_last_diff','give_gini',
 'rec_gini',
 'exc_gini','source+exc',
 'target+exc',
 'exc_ratio',
 'one_way_ratio',
 'first_trade_all',
 'grocery_portion_in_market',
 'money_portion_in_market',
 'other_portion_in_market',
 'first_week_money_by_cnt_dt',
 'money_by_dt_first_max',
 'get_money_cnt_dt_lr',
 'trade_cnt_by_cnt_dt',
 'trade_cnt_by_play_time',
 'trade_cnt_by_combat_time',
 'first_week_money_by_play_time',
 'money_by_time_max_last',
 'money_by_time_first_last',
 'money_by_time_first_max',
 'money_by_time_median_last',
 'get_money_time_lr',
 'last_trade_all',
 'last_week_trade_cnt',
 'first_week_trade_cnt',
 'weekly_max_trade_cnt',
 'weekly_trade_cnt_std',
 'weekly_trade_cycle',
 'trade_first-last',
 'trade_last-max']].to_csv('Features/test_0906_added.csv',index=False)

# 3. Ver2 추가

# Ver2는 다른 문서에서 다루겠음

In [219]:
for i in train_concated.loc[:,sums].isna().sum(1).value_counts().index:
    print('\n',i,'NAs\n',labels.label[train_concated.loc[:,sums].isna().sum(1)==i].value_counts())


 25 NAs
 2month      8799
month       6249
retained    1849
week         261
Name: label, dtype: int64

 29 NAs
 2month      6867
month       4686
retained    3573
week        1212
Name: label, dtype: int64

 10 NAs
 week        7025
month       1530
retained     711
2month       388
Name: label, dtype: int64

 28 NAs
 2month      3956
retained    1844
month       1770
week         342
Name: label, dtype: int64

 15 NAs
 month       2978
week        2492
retained     671
2month       434
Name: label, dtype: int64

 14 NAs
 week        1944
month       1671
retained    1247
2month       671
Name: label, dtype: int64

 9 NAs
 week        2914
retained     862
month        662
2month       463
Name: label, dtype: int64

 16 NAs
 week        1693
month       1423
retained     189
2month       181
Name: label, dtype: int64

 13 NAs
 retained    886
week        749
month       645
2month      453
Name: label, dtype: int64

 2 NAs
 retained    2259
2month       179
month        122
week     